### Memory

In some applications like chatbots, it is important to remember previous interactions to keep the whole context of a conversation. Memory does provide you an easy way to handle this

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello my friend!', additional_kwargs={}, response_metadata={})]

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("hello my friend!")
memory.load_memory_variables({})

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5932\3648818006.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: hi!\nAI: hello my friend!'}

In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain
from pydantic import SecretStr

class MyChatModel(ChatOpenAI):
  def get_num_tokens_from_messages(self, messages):
        # Simple fallback: approximate using string length / 4
        text = "".join(m.content for m in messages)
        return len(text.split())  # crude approximation

llm = MyChatModel(
    model=os.getenv("MODEL", ""),
    base_url=os.getenv("API_URL", ""),
    api_key=SecretStr(os.getenv("API_KEY", "")),
    temperature=0
)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)
conversation.invoke(input={"input": "What is the capital of France?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is the capital of France?
AI:

> Finished chain.


{'input': 'What is the capital of France?',
 'history': '',
 'response': 'The capital of France is **Paris**! 🌟\n\nParis has been the political and cultural heart of France for centuries. It’s home to iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum (where you can see the Mona Lisa). The city is also famous for its charming cafés, world‑class fashion boutiques, and vibrant art scene. Whether you’re strolling along the Seine or exploring the historic Marais district, Paris offers a blend of history, romance, and modernity that makes it truly unique. If you ever plan to visit, don’t miss the chance to try some authentic croissants and maybe catch a sunset from Montmartre!'}

In [6]:
conversation.invoke(input={"input": "Whats the best food there?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the capital of France?
AI: The capital of France is **Paris**.
Human: Whats the best food there?
AI:

> Finished chain.


{'input': 'Whats the best food there?',
 'history': 'Human: What is the capital of France?\nAI: The capital of France is **Paris**.',
 'response': "Ah, you’re in for a culinary adventure! France isn’t just famous for its wine and cheese—though those are absolutely essential—but also for a wide array of dishes that showcase regional flavors, meticulous techniques, and centuries‑old traditions. Here’s a quick tour of some must‑try foods when you’re strolling through Paris (and beyond):\n\n| Dish | What It Is | Why It’s Iconic | Where to Try |\n|------|------------|-----------------|--------------|\n| **Croissant** | Flaky, buttery laminated pastry | Symbol of French breakfast; perfected in Viennese bakeries that migrated to France. | *Le Pain Quotidien* or any local boulangerie on Rue Montorgueil. |\n| **Baguette** | Long, crusty loaf with a crisp exterior and airy interior | The quintessential French bread; the baguette is protected by law (only certain bakers may produce it). | *Poilân

### ConversationSummaryMemory

When inputs get long, we might not want to send the whole conversation, but rather a summary.

In [9]:
from langchain.memory import ConversationSummaryBufferMemory

review = "I ordered Pizza Salami for 9.99$ and it was awesome! \
The pizza was delivered on time and was still hot when I received it. \
The crust was thin and crispy, and the toppings were fresh and flavorful. \
The Salami was well-cooked and complemented the cheese perfectly. \
The price was reasonable and I believe I got my money's worth. \
Overall, I am very satisfied with my order and I would recommend this pizza place to others."

summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) # If Token is 1000, no summary will be created
summary_memory.save_context(
    {"input": "Hello, how can I help you today?"},
    {"output": "Could you analyze a review for me?"},
)
summary_memory.save_context(
    {"input": "Sure, I'd be happy to. Could you provide the review?"},
    {"output": f"{review}"},
)

In [ ]:
conversation = ConversationChain(
    llm=llm, verbose=True, memory=summary_memory
)

conversation.invoke(input="Thank you very much!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, how can I help you today?
AI: Could you analyze a review for me?
Human: Sure, I'd be happy to. Could you provide the review?
AI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.
Human: Thank you very much!
AI:

> Finished chain.


{'input': 'Thank you very much!',
 'history': "Human: Hello, how can I help you today?\nAI: Could you analyze a review for me?\nHuman: Sure, I'd be happy to. Could you provide the review?\nAI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.",
 'response': 'You’re welcome! I’m glad you enjoyed the pizza. If there’s anything else you’d like me to dig into—maybe a deeper sentiment breakdown, comparison with other reviews, or even some suggestions on how to improve your next order—just let me know. I’m here to help!'}

In [11]:
print(summary_memory)

llm=MyChatModel(client=<openai.resources.chat.completions.completions.Completions object at 0x000002134D020A50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002134D0211D0>, root_client=<openai.OpenAI object at 0x000002134D020CD0>, root_async_client=<openai.AsyncOpenAI object at 0x000002134D021450>, model_name='openai/gpt-oss-20b', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://127.0.0.1:1234/v1') chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Thank you very much!', additional_kwargs={}, response_metadata={}), AIMessage(content='You’re welcome! I’m glad you enjoyed the pizza. If there’s anything else you’d like me to dig into—maybe a deeper sentiment breakdown, comparison with other reviews, or even some suggestions on how to improve your next order—just let me know. I’m here to help!', additional_kwargs={}, response_metadata={})]) max_token_limit=100 moving_summary

In [12]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: hello my friend!'}